## Imports and read data

In [1]:
import pandas as pd
import re
from helpers import read_and_describe, calculate_frequent_patterns, generate_association_rules_hybrid
from IPython.display import display, HTML
pd.set_option('display.max_colwidth', None)
pd.set_option('display.max_rows', 37)

import warnings
warnings.filterwarnings("ignore") # ignores warnings

df = read_and_describe('cleaned_data.csv')

Data types:

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 527507 entries, 0 to 527506
Data columns (total 9 columns):
 #   Column       Non-Null Count   Dtype         
---  ------       --------------   -----         
 0   InvoiceNo    527507 non-null  int64         
 1   StockCode    527507 non-null  object        
 2   Description  527507 non-null  object        
 3   Quantity     527507 non-null  int64         
 4   InvoiceDate  527507 non-null  datetime64[ns]
 5   UnitPrice    527507 non-null  float64       
 6   CustomerID   527507 non-null  object        
 7   Country      527507 non-null  object        
 8   TotalPrice   527507 non-null  float64       
dtypes: datetime64[ns](1), float64(2), int64(2), object(4)
memory usage: 36.2+ MB
None

Numeric data description:

           InvoiceNo       Quantity      UnitPrice     TotalPrice
count  527507.000000  527507.000000  527507.000000  527507.000000
mean   559986.722531      10.270061       3.266506      18.992156
std     13435.

### Unifying colors

In [2]:
colors = [
    "Red", "Green", "Blue", "Yellow", "Orange", "Purple", "Pink", "Brown", 
    "Black", "White", "Gray", "Violet", "Indigo", "Magenta", "Cyan", 
    "Turquoise", "Teal", "Lime", "Coral", "Navy", "Maroon", "Olive", 
    "Silver", "Gold", "Beige", "Mint", "Lavender", "Peach", "Ivory", 
    "Charcoal", "Aqua", "Sapphire", "Emerald", "Ruby", "Amber", "CHOCOLATE", 
    "STRAWBERRY", "WOODLAND", "ROSES", "STAR", "HEART"
]

# Regular expression pattern to match colors
color_pattern = r'\b(?:' + '|'.join(colors) + r')\b'

# Function to remove color names
def replace_color_with_colored(description):
    return re.sub(color_pattern, 'COLORED', description, flags=re.IGNORECASE)

# Apply the function to the Description column
df['Description'] = df['Description'].apply(replace_color_with_colored)
df['Description'] = df['Description'].str.strip()
df = df.groupby(['InvoiceNo', 'Description']).agg({
    'Quantity': 'sum', 
    'InvoiceDate': 'first', 
    'StockCode': 'first', 
    'UnitPrice': 'first', 
    'CustomerID': 'first', 
    'Country': 'first',
    'TotalPrice': 'sum'
}).reset_index()

# Frequent patterns for different countries

In [3]:
countries = ['United Kingdom', 'EIRE','Belgium', 'France', 'Germany', 'Netherlands', 'Norway', 'Spain', 'Switzerland', 'Australia']
min_support_values = {
    'United Kingdom': 2.5,
    'EIRE': 7, # Ireland 
    'Belgium': 8,
    'France': 6,
    'Germany': 4,
    'Netherlands': 11,
    'Norway': 13,
    'Spain': 6,
    'Switzerland': 13,
    'Australia': 8.5,
    'Portugal': 15
}

for country in countries:
    min_confidence = 0.5
    confidence_weight = 0.7

    # Display the country name
    display(HTML(f"<div style='font-size: 30px;'>{country}</div>"))

    # Generate and display the association rules
    frequent_patterns = calculate_frequent_patterns(df[df['Country'] == country], min_support_values.get(country)/100)
    rules = generate_association_rules_hybrid(frequent_patterns, min_confidence, confidence_weight)
    rules = rules.sort_values(by='combined_score', ascending=False);
    print(f"Number of rules: {rules.shape[0]}")
    display(rules.reset_index(drop=True).head(10))

Number of rules: 26


,rule,support,confidence,lift,combined_score
0,{'GARDENERS KNEELING PAD CUP OF TEA'} => {'GARDENERS KNEELING PAD KEEP CALM'},0.030207,0.721333,14.386503,0.804933
1,{'CHARLOTTE BAG COLORED POLKADOT'} => {'COLORED RETROSPOT CHARLOTTE BAG'},0.026745,0.711738,14.116541,0.789291
2,{'CHARLOTTE BAG COLORED POLKADOT'} => {'COLORED CHARLOTTE BAG'},0.026186,0.696880,12.735831,0.733239
3,{'GARDENERS KNEELING PAD KEEP CALM'} => {'GARDENERS KNEELING PAD CUP OF TEA'},0.030207,0.602450,14.386503,0.721715
4,{'COLORED RETROSPOT CHARLOTTE BAG'} => {'COLORED CHARLOTTE BAG'},0.032552,0.645626,11.799139,0.666392
5,{'CHARLOTTE BAG SUKI DESIGN'} => {'COLORED CHARLOTTE BAG'},0.028978,0.645522,11.797251,0.666257
6,{'COLORED RETROSPOT CHARLOTTE BAG'} => {'CHARLOTTE BAG COLORED POLKADOT'},0.026745,0.530454,14.116541,0.662392
7,"{'PAPER CHAIN KIT VINTAGE CHRISTMAS'} => {""PAPER CHAIN KIT 50'S CHRISTMAS""}",0.030039,0.672500,10.706200,0.649068
8,{'COLORED CHARLOTTE BAG'} => {'COLORED RETROSPOT CHARLOTTE BAG'},0.032552,0.594898,11.799139,0.630882
9,{'CHARLOTTE BAG SUKI DESIGN'} => {'COLORED RETROSPOT CHARLOTTE BAG'},0.026521,0.590796,11.717782,0.625321


Number of rules: 9


,rule,support,confidence,lift,combined_score
0,{'REGENCY MILK JUG COLORED'} => {'REGENCY SUGAR BOWL COLORED'},0.070922,0.800000,9.024000,0.860000
1,{'REGENCY SUGAR BOWL COLORED'} => {'REGENCY MILK JUG COLORED'},0.070922,0.800000,9.024000,0.860000
2,{'REGENCY SUGAR BOWL COLORED'} => {'COLORED REGENCY TEACUP AND SAUCER'},0.074468,0.840000,4.644706,0.692553
3,{'REGENCY MILK JUG COLORED'} => {'COLORED REGENCY TEACUP AND SAUCER'},0.070922,0.800000,4.423529,0.654682
4,{'REGENCY SUGAR BOWL COLORED'} => {'REGENCY CAKESTAND 3 TIER'},0.074468,0.840000,3.384000,0.636288
5,{'REGENCY TEA PLATE COLORED'} => {'COLORED REGENCY TEACUP AND SAUCER'},0.081560,0.766667,4.239216,0.623122
6,{'REGENCY TEA PLATE COLORED'} => {'REGENCY CAKESTAND 3 TIER'},0.081560,0.766667,3.088571,0.571769
7,{'COLORED REGENCY TEACUP AND SAUCER'} => {'REGENCY CAKESTAND 3 TIER'},0.113475,0.627451,2.527731,0.449288
8,{'SET OF 3 REGENCY CAKE TINS'} => {'REGENCY CAKESTAND 3 TIER'},0.070922,0.571429,2.302041,0.400000


Number of rules: 33


...

,rule,support,confidence,lift,combined_score
0,"{""POPPY'S PLAYHOUSE KITCHEN""} => {""POPPY'S PLAYHOUSE BEDROOM""}",0.079545,1.000000,11.000000,1.000000
1,"{""POPPY'S PLAYHOUSE BEDROOM""} => {""POPPY'S PLAYHOUSE KITCHEN""}",0.079545,0.875000,11.000000,0.912500
2,"{'6 RIBBONS RUSTIC CHARM', 'PACK OF 72 RETROSPOT CAKE CASES'} => {'ASSORTED COLOUR BIRD ORNAMENT'}",0.068182,1.000000,7.333333,0.877778
3,{'SET/5 COLORED RETROSPOT LID GLASS BOWLS'} => {'JAM MAKING SET WITH JARS'},0.068182,1.000000,6.769231,0.858974
4,{'SET OF 6 GIRLS CELEBRATION CANDLES'} => {'SET/10 COLORED POLKADOT PARTY CANDLES'},0.068182,0.857143,9.428571,0.847619
5,"{'ASSORTED COLOUR BIRD ORNAMENT', 'PACK OF 72 RETROSPOT CAKE CASES'} => {'6 RIBBONS RUSTIC CHARM'}",0.068182,1.000000,5.866667,0.828889
6,{'PLASTERS IN TIN CIRCUS PARADE'} => {'PLASTERS IN TIN SKULLS'},0.068182,0.857143,7.542857,0.784762
7,{'SET/10 COLORED POLKADOT PARTY CANDLES'} => {'SET OF 6 GIRLS CELEBRATION CANDLES'},0.068182,0.750000,9.428571,0.772619
8,{'COLORED RETROSPOT TAPE'} => {'6 RIBBONS RUSTIC CHARM'},0.068182,0.857143,5.028571,0.700952
9,{'LUNCH BAG COLORED RETROSPOT'} => {'LUNCH BAG COLORED POLKADOT'},0.068182,0.750000,6.600000,0.678333


Number of rules: 34


,rule,support,confidence,lift,combined_score
0,{'COLORED RETROSPOT BOWL'} => {'COLORED POLKADOT BOWL'},0.16,1.000000,5.000000,1.000000
1,{'COLORED POLKADOT BOWL'} => {'COLORED RETROSPOT BOWL'},0.16,0.800000,5.000000,0.860000
2,{'COLORED TOADSTOOL LED NIGHT LIGHT'} => {'ROUND SNACK BOXES SET OF4 COLORED'},0.16,1.000000,2.941176,0.825000
3,"{'PLASTERS IN TIN CIRCUS PARADE', 'ROUND SNACK BOXES SET OF4 COLORED'} => {'PLASTERS IN TIN SPACEBOY'}",0.14,1.000000,2.777778,0.811111
4,{'PLASTERS IN TIN CIRCUS PARADE'} => {'PLASTERS IN TIN SPACEBOY'},0.20,0.909091,2.525253,0.726010
5,"{'PLASTERS IN TIN SPACEBOY', 'ROUND SNACK BOXES SET OF4 COLORED'} => {'PLASTERS IN TIN CIRCUS PARADE'}",0.14,0.777778,3.535354,0.719949
6,{'ROUND SNACK BOXES SET OF 4 FRUITS'} => {'ROUND SNACK BOXES SET OF4 COLORED'},0.16,0.888889,2.614379,0.719444
7,"{'PLASTERS IN TIN COLORED ANIMALS', 'PLASTERS IN TIN CIRCUS PARADE'} => {'PLASTERS IN TIN SPACEBOY'}",0.16,0.888889,2.469136,0.707099
8,{'PLASTERS IN TIN CIRCUS PARADE'} => {'PLASTERS IN TIN COLORED ANIMALS'},0.18,0.818182,2.406417,0.652273
9,"{'PLASTERS IN TIN CIRCUS PARADE', 'PLASTERS IN TIN SPACEBOY'} => {'PLASTERS IN TIN COLORED ANIMALS'}",0.16,0.800000,2.352941,0.635000


Number of rules: 74


,rule,support,confidence,lift,combined_score
0,"{'DOLLY GIRL LUNCH BOX', 'REGENCY CAKESTAND 3 TIER'} => {'COLORED REGENCY TEACUP AND SAUCER', 'SPACEBOY LUNCH BOX'}",0.089286,1.0,11.200000,1.000000
1,"{'SPACEBOY LUNCH BOX', 'REGENCY CAKESTAND 3 TIER'} => {'COLORED REGENCY TEACUP AND SAUCER', 'DOLLY GIRL LUNCH BOX'}",0.089286,1.0,11.200000,1.000000
2,"{'COLORED REGENCY TEACUP AND SAUCER', 'DOLLY GIRL LUNCH BOX'} => {'SPACEBOY LUNCH BOX', 'REGENCY CAKESTAND 3 TIER'}",0.089286,1.0,11.200000,1.000000
3,"{'COLORED REGENCY TEACUP AND SAUCER', 'SPACEBOY LUNCH BOX'} => {'DOLLY GIRL LUNCH BOX', 'REGENCY CAKESTAND 3 TIER'}",0.089286,1.0,11.200000,1.000000
4,"{'COLORED REGENCY TEACUP AND SAUCER', 'REGENCY CAKESTAND 3 TIER'} => {'DOLLY GIRL LUNCH BOX'}",0.089286,1.0,9.333333,0.927273
5,"{'COLORED REGENCY TEACUP AND SAUCER', 'DOLLY GIRL LUNCH BOX'} => {'REGENCY CAKESTAND 3 TIER'}",0.089286,1.0,9.333333,0.927273
6,"{'CIRCUS PARADE LUNCH BOX', 'DOLLY GIRL LUNCH BOX'} => {'SPACEBOY LUNCH BOX'}",0.089286,1.0,9.333333,0.927273
7,"{'SPACEBOY LUNCH BOX', 'CIRCUS PARADE LUNCH BOX'} => {'DOLLY GIRL LUNCH BOX'}",0.089286,1.0,9.333333,0.927273
8,"{'COLORED REGENCY TEACUP AND SAUCER', 'SPACEBOY LUNCH BOX'} => {'DOLLY GIRL LUNCH BOX'}",0.089286,1.0,9.333333,0.927273
9,"{'COLORED REGENCY TEACUP AND SAUCER', 'DOLLY GIRL LUNCH BOX'} => {'SPACEBOY LUNCH BOX'}",0.089286,1.0,9.333333,0.927273


#### The table above shows a set of association rules per country
#### Each country minimum support has been chosen after try and error mainly to generate a reasonable ammount of itemsets (Not very low, not huge)
#### Minimum confidence is 0.5
#### The association rules was sorted based on a hybrid score between confidence and lift where confidence has 70% weight